This cross-service notebook will walk you through the process of using Textract's DetectDocumenText API to extract text from a JPG/JPEG/PNG file containing text, and then using Comprehend's DetectEntities API to find entities in the extracted text.

You can run this notebook using either AWS Sagemaker or Jupyter's NBViewer tool.

### To run this notebook using Sagemaker

 1. Navigate to Amazon Sagemaker from the AWS Console. From the Sagemaker dashboard, select Notebook and then choose "Notebook instances".
 2. Select the optional "Git repositories" menu, give the notebook instance a name, and choose the option to "Clone a public Git repository to this notebook instance only". 
 3. In the "Git repository URL" box, paste in the URL for this repository (https://github.com/awsdocs/aws-doc-sdk-examples).
 4. Select "Create notebook instance".
 5. You will need to add some policies to your Sagemaker role so it can access S3, Textract, and Comprehend. You can add the following policies to your role: `AmazonTextractFullAccess`, `ComprehendFullAccess`, `AmazonS3ReadOnlyAccess`. 
 6. After the notebook instance has been created, select it from the list of notebooks. 
 7. Choose "Open Jupyter", and after the Jupyter notebook has started navigate to the directory containing this notebook and select this notebook to run it.

### To run this notebook using NBViewer

1. Navigate to the NBViewer tool on Jupyter's website (https://nbviewer.jupyter.org/). Paste the URL of this notebook into the text box on the page and select "Go".
2. On the navigation bar select the Binder symbol to "Execute on Binder". After the notebook build you will be able to run the cells.

In order to make use of the Boto3 Python SDK through NBViewer, you may need to configure your AWS credentials. In the code cell below, replace "KeyID" with the value of your AWS Key ID and replace "AccessKey" with the value of your AWS Secret Access Key. Do not run the following cell if running this notebook in Sagemaker.

In [ ]:
!aws configure set aws_access_key_id "KeyID"
!aws configure set aws_secret_access_key "AccessKey"

After setting your security credentials, you will need to import any libraries you need. You will also need to set the name of both the S3 bucket you have your image in and the name of the image itself. In the code below, replace the value of "bucket-name" with the name of your bucket, replace the value of "document-name" with the name of the image file you want to analyze, and replace the value of "region" with the name of the region you are operating in.

In [ ]:
import boto3
import io
from PIL import Image               
from IPython.display import display 
import json
import pandas as pd

bucket = 'bucket-name'
document = 'document-name'
region = 'region'

You'll need to create a function that connects to both S3 and Textract via the Boto3 SDK. The function presented in the following code starts by connecting to the S3 resource and retrieving the image you specified from the bucket you specified. The function then connects to Textract and calls the DetectDocumentText API to extract the text in the image. The lines of text found in the document are stored in a list and returned.

In [ ]:
# Get the document from S3
s3_connection = boto3.resource("s3")
    
# Connext to Textract to detect text in the document
client = boto3.client("textract", region_name=region)

# Get the rsponse from S3
s3_object = s3_connection.Object(bucket, document)
s3_response = s3_object.get()

# opening binary stream using an in-memory bytes buffer
stream = io.BytesIO(s3_response['Body'].read())

# loading stream into image
image = Image.open(stream)

# Display the image
display(image)
    
def process_text_detection(bucket, document):

    # process using S3 object
    response = client.detect_document_text(
        Document={'S3Object': {'Bucket': bucket, 'Name': document}})

    # Get the text blocks
    blocks = response['Blocks']

    # List to store image lines in document
    line_list = []

    # Create image showing bounding box/polygon the detected lines/text
    for block in blocks:
        if block["BlockType"] == "LINE":
            line_list.append(block["Text"])

    return line_list

lines = process_text_detection(bucket, document)
print("Text found: " + str(lines))

You can now send the lines you extracted from the image to Comprehend and use the service's DetectEntities API to find all entities within those lines. You'll need a function that iterates through the list of lines returned by the "process_text_detection" function you wrote earlier and calls the DetectEntities operation on every line.

In [ ]:

# Connect to comprehend
comprehend = boto3.client(service_name='comprehend', region_name=region)

print('Calling DetectEntities:')
print("------")

def entity_detection(lines):
    
    # Create a list to hold the entities found for every line
    response_entities = []
    
    # Iterate through the lines in the list of lines
    for line in lines:

        # construct a list to hold all found entities for a single line
        entities_list = []

        # Call the DetectEntities operation and pass it a line from lines
        found_entities = comprehend.detect_entities(Text=line, LanguageCode='en')
        for response_data, values in found_entities.items():
            for item in values:
                if "Text" in item:
                    print("Entities found:")
                    for text, val in item.items():
                        if text == "Text":
                            # Append the found entities to the list of entities
                            entities_list.append(val)
                            print(val)
        # Add all found entities for this line to the list of all entities found
        response_entities.append(entities_list)

    return response_entities

response_ents = entity_detection(lines)

Now that you have a list of the lines extracted by Textract and the entities found in those lines, you can create a dataframe that lets you see both. In the code below, a Pandas dataframe is constructed, displaying the lines found in the input image and their associated entities.

In [ ]:
entities_dict = {"Lines":lines, "Entities":response_ents}
df = pd.DataFrame(entities_dict, columns=["Lines","Entities"])
print(df)